# 統合モデル学習のための観測情報取得

## パラメータ
各パラメータをデータに合わせて設定

In [1]:
# GP-HSMM
CLASS_NUM = 7 #動作の最大クラス数
OBJECT_NUM = 3 # 物体の種類(GP-HSMMの種類)

# OBJECT
CATEGORY_NUM = 3 # 物体の種類(物体概念用)

# plase
Stick_large_L=5 # 場所概念の学習時に設定したスティックブレーキング過程での最大カテゴリ数

# woed
WORD_MAX_DIM = 200 # 単語のBoWの次元数

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mmlda_bhmm.word_client import WordClient
import os

In [3]:
FLD="data/"
DATA = "exp_crest_08_{0:d}/"
SEQS = "exp_crest_08_{0:d}_sequence.csv"
HAND = "exp_crest_08_{0:d}_hand_state.csv"
OBJ = "object_data_frame_dr01_0801_4.csv"
NUM = [0,1,2,3,4,5,6,7,8,12,13,14,15,17]

In [4]:
dfs = [pd.read_csv(FLD+DATA.format(i)+HAND.format(i)) for i in NUM]
dfseqs = [pd.read_csv(FLD+DATA.format(i)+SEQS.format(i)) for i in NUM]
obj = pd.read_csv(FLD+OBJ)

In [5]:
LEARN = "learn/"+"1/"
CAT = ["box","pettbotle","door"]
STAMP = "/stamps{0:03d}.txt"
SLEN = "/slen{0:03d}.txt"

In [6]:
datas = []
clens = []
for c in CAT:
    sts = []
    sls = []
    i = 0
    while 1:
        try:
            st = np.loadtxt(LEARN+c+STAMP.format(i))
        except:
            break
        sl = np.loadtxt(LEARN+c+SLEN.format(i))
        stls = [st[0,0],st[-1,1]]
        slls = [int(sl[j,0]) for j in range(len(sl))]
        sts.append(stls)
        sls.append(slls)
        i += 1
    datas.append(sts)
    clens.append(sls)

In [7]:
seqs = []
for i in range(len(CAT)):
    data = datas[i]
    seq = []
    for j in range(len(data)):
        dd = data[j]
        for dfseq in dfseqs:
            d = dfseq.loc[((dfseq.time >= dd[0])&(dfseq.time <= dd[1]))]
            if len(d)==0:
                continue
            else:
                s = d.sentence.values
            seq.append(s)
    seqs.append(seq)


In [8]:
try:
    os.mkdir(LEARN+"caption")
except:
    pass

In [9]:
time = []
sentence = []
cat = []
act = []
for i in range(len(CAT)):
    for j in range(len(datas[i])):
        d = datas[i][j][0]
        sen = list(seqs[i][j])
        time.append(d)
        sentence.append(sen)
        cat.append(i)
        act.append(clens[i][j])

In [10]:
df = pd.DataFrame()
df["time"] = time
df["sentence"] = sentence
df["category"] = cat
df["action"] = act

In [11]:
df = df.sort_values("time")
df = df.reset_index()
df = df.drop(["index"],axis=1)
df.head()

,time,sentence,category,action
0,1.533107e+09,"[キッチンでペットボトルを掴む, キッチンでペットボトルを掴む, キッチンでペットボトルを掴む]",1,"[3, 3, 2, 2, 2, 2, 0, 0]"
1,1.533107e+09,"[リビングで箱に入れる, リビングで箱に入れる]",0,"[3, 3, 3, 0, 1]"
2,1.533107e+09,"[リビングで箱に入れる, リビングで箱に入れる]",0,"[2, 3, 3, 3, 3, 1]"
3,1.533107e+09,"[キッチンでペットボトルを掴む, キッチンでペットボトルを掴む]",1,"[3, 3, 3, 3, 3, 3, 3, 1, 0, 0]"
4,1.533107e+09,"[リビングで箱に入れる, リビングで箱に入れる]",0,"[3, 0, 1]"


### エピソード数を追記

In [12]:
t = df.time.values
task = [0] * len(t)
for ti in NUM:
    d = pd.read_csv(FLD+DATA.format(ti)+HAND.format(ti))
    _t = d.time.values
    for j in range(len(t)):
        if t[j] >= _t[0] and t[j] <= t[-1]:
            task[j] = ti
df["episode"] = task

### ステップ数を追記

In [13]:
steps = []
for e in df.episode.unique():
    step_size = len(df[df.episode == e])
    steps.extend(range(step_size))
df["step"] = steps

### 場所情報の追加

rosbag2df

In [16]:
import sys
sys.path.append("/opt/ros/indigo/lib/python2.7/dist-packages/")
import rosbag
from geometry_msgs.msg import PoseWithCovarianceStamped
import glob

No handlers could be found for logger "rosout"


In [17]:
rosbag_dir = "../../../data/crest_demo_rosbag/"
rosbag_path_list = glob.glob(os.path.join(rosbag_dir,"*"))
rosbag_path_list.sort()

In [18]:
def poose2xysc(msg):
    pose = msg.pose.pose.position
    orientation = msg.pose.pose.orientation
    sin = 2 * orientation.w * orientation.z
    cos = orientation.w * orientation.w - orientation.z * orientation.z
    return pose.x, pose.y, sin, cos

rosbag_df = pd.DataFrame(columns=["time","x","y","sin","cos"])
for rosbag_path in rosbag_path_list:
    bag = rosbag.Bag(rosbag_path)
    values = []
    for topic, msg, t in bag.read_messages('/laser_2d_pose'):
        x, y, sin, cos = poose2xysc(msg)
        time = msg.header.stamp.to_nsec()/ float(pow(10,9))
        values.append([time, x, y, sin, cos])
    tmp_df = pd.DataFrame(data=values, columns=["time","x","y","sin","cos"])
    rosbag_df = rosbag_df.append(tmp_df, ignore_index=True)
rosbag_df = rosbag_df.sort_values(by="time")
rosbag_df.to_csv("../src/place_xysc.csv")

In [19]:
rosbag_df = pd.read_csv("../src/place_xysc.csv")

timestampを使用して、場所とそれ以外のモダリティを結合  
場所以外の情報を取得した時間以上で最も早いものを同じstepのデータとする．

In [20]:
place_df = df[["episode","step","time"]].sort_values(by="time")
place_df.loc[:,"x"] = None
place_df.loc[:,"y"] = None
place_df.loc[:,"sin"] = None
place_df.loc[:,"cos"] = None
# time_df = time_df.set_index(["episode","step","time"])

In [21]:
for e in place_df.episode.unique():
    for s in place_df[place_df.episode==e].step.unique():
        tmp_time = place_df[(place_df.episode==e) & (place_df.step==s)].time.values.flatten()[0]
        tmp_df = rosbag_df[rosbag_df.time >= tmp_time].iloc[0]
        place_df.loc[(place_df.episode==e) & (place_df.step==s), "x"] = tmp_df.x
        place_df.loc[(place_df.episode==e) & (place_df.step==s), "y"] = tmp_df.y
        place_df.loc[(place_df.episode==e) & (place_df.step==s), "sin"] = tmp_df.sin
        place_df.loc[(place_df.episode==e) & (place_df.step==s), "cos"] = tmp_df.cos
place_df = place_df.drop("time",axis=1)

In [22]:
place_df.head()

,episode,step,x,y,sin,cos
0,0,0,-0.188579,-1.43097,-0.998993,0.0448609
1,0,1,-1.89124,0.729737,-0.0708271,-0.997489
2,1,0,-0.682719,0.547449,0.100007,-0.994987
3,2,0,0.0365419,-1.40244,-0.99953,0.0306651
4,2,1,-0.901168,0.666818,0.0278298,-0.999613


### place用pandasの作成

場所の位置情報から場所概念の情報に変換

In [23]:
import re
from scipy.stats import multivariate_normal

PARAMETER_PATH = "../../spco_formation/result/trial"

def xysc2pconcept(pose_x,pose_y,sin,cos):

    mu = []
    file = glob.glob(PARAMETER_PATH + '/mu/*')
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    file.sort(key = alphanum_key)
    for f in file:
        hoge = np.loadtxt(f)
        mu.append(hoge)

    sigma = []
    file = glob.glob(PARAMETER_PATH + "/sigma/*.csv")
    file.sort(key = alphanum_key)
    for f in file:
        hoge = np.loadtxt(f)
        sigma.append(hoge)
        
    pi = np.loadtxt(PARAMETER_PATH + "/pi.csv")

    cordinate=[float(pose_x),float(pose_y),float(sin),float(cos)]
    r_prob=np.ones(len(mu))
    for z in xrange(len(mu)):
        r_prob[z] = multivariate_normal.pdf(cordinate, mean=mu[z], cov=sigma[z])
        r_prob[z] = r_prob[z] * pi[z]

    return r_prob /r_prob.sum()

In [24]:
place_bow_df = pd.DataFrame()
for i,row in place_df.iterrows():
    c_prob = xysc2pconcept(row.x, row.y, row.sin, row.cos)
    place_clase = np.argmax(c_prob)
    place_hist = np.eye(Stick_large_L)[place_clase]
    value = np.insert(place_hist,0,[row.episode,row.step])
    tmp_df = pd.DataFrame([value],dtype=int)
    place_bow_df = pd.concat([place_bow_df,tmp_df],axis=0)
place_bow_df.columns = ["episode","step"] + [("place_hist",i) for i in range(Stick_large_L)]
place_bow_df.index = range(len(place_bow_df))

### sentences用pandasの作成

In [25]:
sen_df = pd.DataFrame()
for i,row in df.iterrows():
    value = []
    for sen in row.sentence:
        value.append([sen,row.episode,row.step])
    tmp_df = pd.DataFrame(value)
    sen_df = pd.concat([sen_df,tmp_df],axis=0)
sen_df.columns = ["sentence","episode","step"]
sen_df.index = range(len(sen_df))

# sentences2bow
word_client = WordClient()
word_client.num_dim = WORD_MAX_DIM # 単語の最大次元数

# word_client.load_codebook("./codebook.txt")

word_all = []
for e in sen_df.episode.unique():
    for s in sen_df[sen_df.episode==e].step.unique():
        word_client.setAccept()
        for sen in sen_df[(sen_df.episode==e) & (sen_df.step==s)].sentence.values:
                word_client.sentence_data_cb(sen)
        word_client.setReject()
        word_client.split_sentence()
        word_client.update_word_codebook()
        word_client.words2histogram()
        word_histogram = word_client.get_histgram()
        word_client.reset()
        word_all.append(word_histogram)

word_df = pd.DataFrame(np.array(word_all),dtype=int)

word_df = pd.concat([df[["episode","step"]].astype(int),word_df],axis=1)
word_df = word_df.astype(int)

word_df.columns = ["episode","step"] + [("word_hist",i) for i in range(word_client.num_dim)]
word_df.index = range(len(word_df))
# word_df = word_df.set_index(["episode","step"])
# word_df.columns = pd.MultiIndex.from_tuples(word_df.columns)

### action用 pandasの作成

In [35]:
class_nums = [CLASS_NUM for i in range(OBJECT_NUM)] # class num of gp-hsmm
START_CLASS = CLASS_NUM
END_CLASS = CLASS_NUM + 1

def seq2bow(seq, object_id):
    """
    GP-HSMMのシーケンス情報をbowに変換する
    seq: gp-hsmmの状態
    object_id: objectのcategory
    """
    motions = np.insert(seq,0,[START_CLASS,END_CLASS])
    motions = motions +  object_id * (END_CLASS + 1)
    motion_hists = []
    for i in range((END_CLASS+1) * len(class_nums)):
        motion_hists.append(len(np.where(motions==i)[0]))
    return motion_hists

action_df = pd.DataFrame()
for i,row in df.iterrows():
    action_hist = seq2bow(row.action,row.category)
    value= np.insert(action_hist,0,[row.episode,row.step])
    tmp_df = pd.DataFrame([value])
    action_df = pd.concat([action_df,tmp_df],axis=0)
action_df.columns = ["episode","step"] + [("action_hist",i) for i in range((END_CLASS+1) * len(class_nums))]
action_df.index = range(len(action_df))

# action_df = action_df.set_index(["episode","step"])
# action_df.columns = pd.MultiIndex.from_tuples(action_df.columns)


### category(object)用 pandasの作成

In [38]:
category_df = pd.DataFrame()
for i,row in df.iterrows():
    category_hist = np.eye(CATEGORY_NUM,dtype=int)[row.category]
    value = np.insert(category_hist,0,[row.episode,row.step])
    tmp_df = pd.DataFrame([value])
    category_df = pd.concat([category_df,tmp_df],axis=0)
category_df.columns = ["episode","step"] + [("category_hist",i) for i in range(CATEGORY_NUM)]
category_df.index = range(len(category_df))

# category_df = category_df.set_index(["episode","step"])
# category_df.columns = pd.MultiIndex.from_tuples(category_df.columns)

### 全情報[time, word, object, action, place]の情報を結合

In [39]:
obs_df = df[["episode","step","time"]]
obs_df = pd.merge(obs_df,word_df,on=["episode","step"])
obs_df = pd.merge(obs_df,category_df,on=["episode","step"])
obs_df = pd.merge(obs_df,action_df,on=["episode","step"])
obs_df = pd.merge(obs_df,place_bow_df,on=["episode","step"])

obs_df = obs_df.set_index(["episode","step","time"])
obs_df.columns = pd.MultiIndex.from_tuples(obs_df.columns)
obs_df

word_hist                                            \
                                 0  1  2  3  4  5  6  7  8  9  10 11 12 13 14   
episode step time                                                               
0       0    1.533107e+09         3  3  3  3  3  0  0  0  0  0  0  0  0  0  0   
        1    1.533107e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
1       0    1.533107e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
2       0    1.533107e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        1    1.533107e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
3       0    1.533108e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
        1    1.533108e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
4       0    1.533108e+09         0  1  0  0  0  1  1  1  1  0  0  0  0  0  0   
5       0    1.533108e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
6       0    1.533108e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        1    1.533108e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
7       0    1.533108e+09         0  2  0  2  0  0  0  0  0  4  2  2  0  0  0   
        1    1.533109e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        2    1.533109e+09         0  3  0  0  0  3  3  3  3  0  0  0  0  0  0   
8       0    1.533109e+09         0  2  0  2  0  0  0  0  0  4  2  2  0  0  0   
        1    1.533109e+09         0  1  0  0  0  1  1  1  1  0  0  0  0  0  0   
12      0    1.533110e+09         0  2  0  2  0  0  0  0  0  4  2  2  0  0  0   
        1    1.533110e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        2    1.533110e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
13      0    1.533110e+09         0  3  0  3  0  0  0  0  0  6  3  3  0  0  0   
        1    1.533110e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        2    1.533110e+09         0  1  0  0  0  1  1  1  1  0  0  0  0  0  0   
14      0    1.533111e+09         0  2  0  2  0  0  0  0  0  4  2  2  0  0  0   
        1    1.533111e+09         2  2  2  2  2  0  0  0  0  0  0  0  0  0  0   
        2    1.533111e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   
15      0    1.533111e+09         0  3  0  3  0  0  0  0  0  6  3  3  0  0  0   
        1    1.533111e+09         0  3  0  0  0  3  3  3  3  0  0  0  0  0  0   
17      0    1.533111e+09         0  2  0  2  0  0  0  0  0  4  2  2  0  0  0   
        1    1.533111e+09         3  3  3  3  2  0  0  0  0  0  0  0  1  0  0   
        2    1.533111e+09         0  2  0  0  0  2  2  2  2  0  0  0  0  0  0   

                                                        ... action_hist        \
                          15 16 17 18 19 20 21 22 23 24 ...          7  8  9    
episode step time                                       ...                     
0       0    1.533107e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  2   
        1    1.533107e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
1       0    1.533107e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
2       0    1.533107e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  2   
        1    1.533107e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
3       0    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
        1    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  4   
4       0    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
5       0    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
6       0    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  3   
        1    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           1  1  0   
7       0    1.533108e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  0   
        1    1.533109e+09  0  0  0  0  0  0  0  0  0  0 ...           0  0  2   
        2    1.533109e+09  0  0  0  0  0  0  0  0  0  0 

保存

In [40]:
obs_df.to_csv("../src/obs_data.csv")
sen_df.to_csv("../src/sen_data.csv",index=False)

### serket入力用の書式に変換(テスト用)

In [41]:
def df2list(tmp_df, hist_name):
    tmp_hists = []
    tmp_df = tmp_df.loc[:,hist_name]
    for i,row in tmp_df.iterrows():
        tmp_hists.append(row.values)
    return np.array(tmp_hists)

def make_obs(obs_data, sen_data):
    # データの成形
    object_object = df2list(obs_data,"category_hist")
    motion_motion = df2list(obs_data,"action_hist")
    place_place = df2list(obs_data,"place_hist")
    words_all =  df2list(obs_df,"word_hist")  
    splited_sentences = []    

    # 全データを貯める
    
    # word の処理
    word_client.setAccept()
    for _,sen in sen_data.iterrows():
        word_client.sentence_data_cb(sen.sentence)
    word_client.setReject()
    word_client.split_sentence()
    word_client.update_word_codebook()
    word_client.words2histogram()
    splited_sen = word_client.get_splited_sentences()
    word_histogram = word_client.get_histgram()
    word_client.reset()
    splited_sentences.extend(splited_sen)
    
    codebook = word_client._codebook
    obs = [object_object, motion_motion,
    place_place, words_all, 
    np.array(splited_sentences), codebook]
    correct = [None,None,None]
    # correct = [dataframe.state.values, dataframe.action.values, dataframe.place.values]
    return obs, correct

In [42]:
obs = make_obs(obs_df, sen_df)